# 10 – Anti-Deception Analysis

In this notebook, we explore whether deceptive body movements influence prediction difficulty.
We aim to:
- Identify 'uncertain' sequences (low model confidence)
- Highlight misclassified examples with feints or body sways
- Visualize keypoints for difficult vs. easy predictions
- Discuss implications for goalkeeper training

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# y_true, y_pred, y_pred_classes disponibles depuis notebook 04 ou 05
# X : séquences de keypoints normalisées ou brutes
# target_names = ['g', 'm', 'd']

In [ ]:
# Identifier les prédictions les moins sûres (basées sur la probabilité max)
confidences = np.max(y_pred, axis=1)
low_conf_indices = np.argsort(confidences)[:10]
print("Séquences les plus ambiguës :")
for i in low_conf_indices:
    print(f"→ {i}: confidence={confidences[i]:.2f}, true={target_names[y_true[i]]}, pred={target_names[y_pred_classes[i]]}")

In [ ]:
# Visualiser les poses des cas incertains
def plot_skeleton(sequence, title=""):
    frame = sequence[-1]  # dernière frame avant tir
    xs = frame[::2]
    ys = frame[1::2]
    plt.figure()
    plt.scatter(xs, -ys)
    plt.title(title)
    plt.axis('equal')

for i in low_conf_indices[:3]:
    plot_skeleton(X[i], f"Conf={confidences[i]:.2f} | True={target_names[y_true[i]]} | Pred={target_names[y_pred_classes[i]]}")

In [ ]:
# Distribution des confiances selon que la prédiction est correcte ou non
correct = (y_true == y_pred_classes)
plt.hist(confidences[correct], bins=20, alpha=0.6, label='Correct')
plt.hist(confidences[~correct], bins=20, alpha=0.6, label='Incorrect')
plt.xlabel('Confidence')
plt.ylabel('Count')
plt.title('Confidence Distribution')
plt.legend()
plt.show()

In [ ]:
# Hypothèse : les séquences trompeuses ont des variations plus importantes (pose 'instable')
variances = np.array([np.var(seq) for seq in X])
plt.scatter(confidences, variances, c=correct, cmap='coolwarm', alpha=0.7)
plt.xlabel('Confidence')
plt.ylabel('Pose Variance')
plt.title('Variance vs Confidence')
plt.colorbar(label='Correct (1) or Incorrect (0)')
plt.show()